In [1]:
import os, sys, glob
import numpy as np
import scipy.signal as sigpro
import scipy.linalg as scpla

from tqdm.notebook import tqdm
import math
import string

import heapq
from collections import defaultdict
import itertools
from functools import lru_cache



%matplotlib notebook
import matplotlib.pyplot as plt

In [60]:
def dprint(lvl, msg, end='\n'):
    if dbg>=lvl:
        print(msg, end=end)

In [22]:
### Problem 1

txtin = ['1000',
    '2000',
    '3000',
    '',
    '4000',
    '',
    '5000',
    '6000',
    '',
    '7000',
    '8000',
    '9000',
    '',
    '10000']

with open('p01_input.txt', 'r') as fid:
    ptvals = fid.readlines()
#print(ptvals)

def ptext(txtin):
    txtvals = [{'tot':0, 'vals':[]}]
    for ln in txtin:
        if '\n' in ln:
            ln = ln.replace('\n', '')
        if ln == '':
            txtvals.append({'tot':0, 'vals':[]})
        else:
            v = int(ln)
            txtvals[-1]['tot'] += v
            txtvals[-1]['vals'].append(v)
    vv = np.array([v['tot'] for v in txtvals])
    vs = np.sort(vv)
    vi = np.argsort(vv)
    print('Max {} for elf {}, top 3 elves have {}'.format(vs[-1], 1+vi[-1], np.sum(vs[-3:])))
    
    return txtvals

vo = ptext(txtin)
vo = ptext(ptvals)


Max 24000 for elf 4, top 3 elves have 45000
Max 74198 for elf 209, top 3 elves have 209914


In [28]:
### Problem 2

txtin = ['A Y',
            'B X',
            'C Z']

with open('p02_input.txt', 'r') as fid:
    ptvals = fid.readlines()
    
def p2scoreit(txvals):
    scr = {'rock': 1, 'paper':2, 'scissors':3}
    emap = {'A':'rock', 'B':'paper', 'C':'scissors'}
    mmap = {'X':'rock', 'Y':'paper', 'Z':'scissors'}
    
    wins = ['rs','pr','sp']
    
    wscr = [0, 3, 6]
    totscr = 0
    for ln in txvals:
        ln = ln.replace('\n','')
        thisround = mmap[ln[-1]][0]+emap[ln[0]][0]
        thisscore = scr[mmap[ln[-1]]]
        if thisround in wins:
            thisscore += 6
        elif thisround[0] == thisround[-1]:
            thisscore += 3
        #print(thisround, thisscore)
        totscr += thisscore
    return totscr

def p2bscoreit(txvals):
    scr = {'r': 1, 'p':2, 's':3}
    emap = {'A':'r', 'B':'p', 'C':'s'}
    mmap = {'X':'lose', 'Y':'draw', 'Z':'win'}
    
    wins = ['rs','pr','sp']
    
    plays = {'win':{'r':'p', 'p':'s', 's':'r'},
             'draw':{'r':'r', 'p':'p', 's':'s'},
             'lose':{'r':'s', 'p':'r', 's':'p'}}
    
    wscr = [0, 3, 6]
    totscr = 0
    for ln in txvals:
        ln = ln.replace('\n','')
        thisround = plays[mmap[ln[-1]]][emap[ln[0]]] + emap[ln[0]]
        thisscore = scr[thisround[0]]
        if thisround in wins:
            thisscore += 6
        elif thisround[0] == thisround[-1]:
            thisscore += 3
        #print(thisround, thisscore)
        totscr += thisscore
    return totscr

print(p2scoreit(ptvals))
print(p2bscoreit(ptvals))
        


11603
12725


In [32]:
### Problem 3

import string


txtin = ['vJrwpWtwJgWrhcsFMMfFFhFp',
        'jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL',
        'PmmdzqPrVvPwwTWBwg',
        'wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn',
        'ttgJtRGJQctTZtZT',
        'CrZsJsPPZsGzwwsLwLmpwMDw']

with open('p03_input.txt', 'r') as fid:
    ptvals = fid.readlines()
def get_pri(textin):
    pri = {}
    p = 0
    for char in string.ascii_lowercase:
        p+=1
        pri[char] = p
    for char in string.ascii_uppercase:
        p+=1
        pri[char] = p
    pscore=0
    for ln in textin:
        ln = ln.replace('\n', '')
        N = len(ln)
        for ii in range(N//2):
            if ln[ii] in ln[N//2:]:
                pscore += pri[ln[ii]]
                break
    print('Score = {}'.format(pscore))
    
def get_badge_pri(textin):
    pri = {}
    p = 0
    for char in string.ascii_lowercase:
        p+=1
        pri[char] = p
    for char in string.ascii_uppercase:
        p+=1
        pri[char] = p
    pscore=0
    for lnx in range(len(textin)//3):
        ln = textin[3*lnx].replace('\n', '')
        N = len(ln)
        for ii in range(N):
            if ln[ii] in textin[3*lnx+1] and ln[ii] in textin[3*lnx+2]:
                pscore += pri[ln[ii]]
                break
    print('Score = {}'.format(pscore))
get_pri(txtin)
get_pri(ptvals)

get_badge_pri(txtin)
get_badge_pri(ptvals)

Score = 157
Score = 7428
Score = 70
Score = 2650


In [39]:



txtin = ['2-4,6-8',
        '2-3,4-5',
        '5-7,7-9',
        '2-8,3-7',
        '6-6,4-6',
        '2-6,4-8']

with open('p04_input.txt', 'r') as fid:
    ptvals = fid.readlines()
    
def count_overlap_pairs(textvals):
    ctp = 0
    ctq = 0
    
    for ln in textvals:
        ln = ln.replace('\n', '')
        
        lnsplt = ln.split(',')
        lna = lnsplt[0].split('-')
        #print(lna)
        lnb = lnsplt[1].split('-')
        #print(lnb)
        r1 = [int(lna[0]), int(lna[1])]
        r2 = [int(lnb[0]), int(lnb[1])]
        
        if (r1[0]>=r2[0] and r1[1]<=r2[1]) or (r2[0]>=r1[0] and r2[1]<=r1[1]):
            ctp += 1
            
        if (r1[0]>=r2[0] and r1[0]<=r2[1]) or (r2[0]>=r1[0] and r2[0]<=r1[1]):
            ctq += 1
    print('{} full overlapping pairs'.format(ctp))
    
    print('{} partial overlapping pairs'.format(ctq))
count_overlap_pairs(txtin)
count_overlap_pairs(ptvals)
        



2 full overlapping pairs
4 partial overlapping pairs
500 full overlapping pairs
815 partial overlapping pairs


In [63]:
txtin = ['    [D]    ',
         '[N] [C]    ',
         '[Z] [M] [P]',
         ' 1   2   3 ',
         '',
         'move 1 from 2 to 1',
         'move 3 from 1 to 3',
         'move 2 from 2 to 1',
         'move 1 from 1 to 2']

with open('p05_input.txt', 'r') as fid:
    ptvals = fid.readlines()

dbg = 0

def restack(textin):
    #read the current stacks
    
    stacks = True
    cstk = []
    for ln in textin:
        ln = ln.replace('\n', '')
        dprint(1,ln)
        if len(ln) == 0:
            stacks = False
            dprint(1,cstk)
            continue
        if stacks:
            if len(cstk) == 0:
                numstacks = (1+len(ln))//4
                for ii in range(numstacks):
                    cstk.append('')
            for ii in range(numstacks):
                iu = 4*ii+1
                if ln[iu] == '1':
                    break
                if ln[iu] != ' ':
                    cstk[ii] = cstk[ii]+ln[iu]
        else:
            #dprint(c1,stk)
            num2move = int(ln.split(' ')[1])
            mvfrom = int(ln.split(' ')[3])
            mvto = int(ln.split(' ')[5])
            for ii in range(num2move):
                cstk[mvto-1] = cstk[mvfrom-1][0] + cstk[mvto-1]
                cstk[mvfrom-1] = cstk[mvfrom-1][1:]
            dprint(1,cstk)
            dprint(1,'=='*30)
    for stk in cstk:
        print(stk[0], end='')
    print()
def restack9001(textin):
    #read the current stacks
    
    stacks = True
    cstk = []
    for ln in textin:
        ln = ln.replace('\n', '')
        dprint(1,ln)
        if len(ln) == 0:
            stacks = False
            dprint(1,cstk)
            continue
        if stacks:
            if len(cstk) == 0:
                numstacks = (1+len(ln))//4
                for ii in range(numstacks):
                    cstk.append('')
            for ii in range(numstacks):
                iu = 4*ii+1
                if ln[iu] == '1':
                    break
                if ln[iu] != ' ':
                    cstk[ii] = cstk[ii]+ln[iu]
        else:
            #print(cstk)
            num2move = int(ln.split(' ')[1])
            mvfrom = int(ln.split(' ')[3])
            mvto = int(ln.split(' ')[5])
            
            cstk[mvto-1] = cstk[mvfrom-1][:num2move] + cstk[mvto-1]
            cstk[mvfrom-1] = cstk[mvfrom-1][num2move:]
            dprint(1,cstk)
            dprint(1,'=='*30)
    for stk in cstk:
        print(stk[0], end='')
    print()            
    
restack(txtin)
restack(ptvals)

restack9001(txtin)
restack9001(ptvals)

    
    

CMZ
QMBMJDFTD
MCD
NBTVTJNFJ


In [79]:
txtin = ['mjqjpqmgbljsphdztnvjfqwrcgsmlb',# seven
         'bvwbjplbgvbhsrlpgdmjqwftvncz',#: first marker after character 5
         'nppdvjthqldpwncqszvftbrmjlhg', #: first marker after character 6
         'nznrnfrfntjfmvfwmzdfjlvtqnbhcprsg', #: first marker after character 10
         'zcfzfwzzqfrljwzlrfnpqdbhtmscgvjw'] #: first marker after character 11',

with open('p06_input.txt', 'r') as fid:
    ptvals = fid.readlines()
#print(ptvals)
def find_marker(textin):
    txu = textin.replace('\n','')
    tused = ''
    gct = 0
    for itx, t in enumerate(txu):
        if itx>=3:
            tused = t
            for tt in txu[itx-1:itx-4:-1]:
                if not tt in tused:
                    tused = tused+tt
            if len(tused) == 4:
                #print(txu[:itx+1])
                #print('{} + {}'.format(txu[itx-3:itx], t))
                print('Marker at {}'.format(itx+1))
                break
def find_fmarker(textin):
    txu = textin.replace('\n','')
    tused = ''
    gct = 0
    for itx, t in enumerate(txu):
        if itx>=13:
            tused = t
            for tt in txu[itx-1:itx-14:-1]:
                if not tt in tused:
                    tused = tused+tt
            if len(tused) == 14:
                #print(txu[:itx+1])
                #print('{} + {}'.format(txu[itx-3:itx], t))
                print('Marker at {}'.format(itx+1))
                break

for tcheck in txtin:
    find_marker(tcheck)

find_marker(ptvals[0])

find_fmarker(ptvals[0])

Marker at 7
Marker at 5
Marker at 6
Marker at 10
Marker at 11
Marker at 1625
Marker at 2250


In [121]:
txtin = ['$ cd /',
        '$ ls',
        'dir a',
        '14848514 b.txt',
        '8504156 c.dat',
        'dir d',
        '$ cd a',
        '$ ls',
        'dir e',
        '29116 f',
        '2557 g',
        '62596 h.lst',
        '$ cd e',
        '$ ls',
        '584 i',
        '$ cd ..',
        '$ cd ..',
        '$ cd d',
        '$ ls',
        '4060174 j',
        '8033020 d.log',
        '5626152 d.ext',
        '7214296 k']

with open('p07_input.txt', 'r') as fid:
    ptvals = fid.readlines()

def get_dir_info(textin):
    ds = {}
    cwd = None
    mxlvl = 0
    
    for ln in textin:
        ln = ln.replace('\n','')
        if ln.startswith('$ '):
            # print(ln)
            lncmd = ln.split(' ')
            if lncmd[1] == 'cd':
                if lncmd[-1] == '..':
                    owd = cwd
                    cwd = ds[cwd]['parent']
                    ds[cwd]['size'] += ds[owd]['size']
                    lvl -= 1
                elif lncmd[-1] == '/':
                    cwd = lncmd[-1]
                    ds[cwd] = {'parent':None, 'level':0, 'size':0, 'files':[], 'children':[]}
                    lvl = 0
                else:
                    lvl += 1
                    owd = cwd
                    if owd is None:
                        cwd = lncmd[-1]
                    else:
                        cwd = owd+'-'+lncmd[-1]
                    if cwd in ds.keys():
                        print('We seem to be revisiting')
                    else:
                        ds[cwd] = {'parent':owd, 'level':ds[owd]['level']+1, 'size':0, 'files':[], 'children':[]}
                        if ds[cwd]['level'] > mxlvl:
                            mxlvl = ds[cwd]['level']
            else:
                ajnk = 0
                #print('Doing an ls {} in {}'.format(lncmd[-1], cwd))
        else:
            lninfo = ln.split(' ')
            if lninfo[0].startswith('dir'):
                ds[cwd]['children'].append(lninfo[-1])
            else:
                sz = int(lninfo[0])
                ds[cwd]['files'].append({'size':sz, 'name':lninfo[-1]})
                ds[cwd]['size'] += sz
    while lvl > 0:
        owd = cwd
        cwd = ds[cwd]['parent']
        ds[cwd]['size'] += ds[owd]['size']
        lvl -= 1
        
    
    
    allsize = 0    
    for ky in ds.keys():
        if ds[ky]['size'] < 100000:
            allsize += ds[ky]['size']
    print('{} total in directories less the 100K'.format(allsize))
    # print(ds)  
    # find the best directory to delete
    
    totsize = ds['/']['size']
    ktarget = totsize - (70000000-30000000)
    
    ktc = 5000*ktarget
    print(totsize,ktarget, ktc)
    
    for ky in ds.keys():
        if ds[ky]['size'] >= ktarget:
            
            if ds[ky]['size'] < ktc:
                ktc = ds[ky]['size']
    print('Need to delete: {}'.format(ktc))
            
            
        
        
        
get_dir_info(txtin)
print(txtin)    
#print(ptvals)
print('- '*30)
get_dir_info(ptvals)

95437 total in directories less the 100K
48381165 8381165 41905825000
Need to delete: 24933642
['$ cd /', '$ ls', 'dir a', '14848514 b.txt', '8504156 c.dat', 'dir d', '$ cd a', '$ ls', 'dir e', '29116 f', '2557 g', '62596 h.lst', '$ cd e', '$ ls', '584 i', '$ cd ..', '$ cd ..', '$ cd d', '$ ls', '4060174 j', '8033020 d.log', '5626152 d.ext', '7214296 k']
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
1555642 total in directories less the 100K
45349983 5349983 26749915000
Need to delete: 5974547


In [135]:
txtin = ['30373',
         '25512',
         '65332',
         '33549',
         '35390']

with open('p08_input.txt', 'r') as fid:
    ptvals = fid.readlines()
    
def vmap(textin):
    nrows = len(textin)
    ln0 = textin[0].replace('\n','')
    ncols = len(ln0)
    themap = np.zeros((nrows, ncols), dtype = 'int')
    #convert the text to a grid
    for lnx, ln in enumerate(textin):
        ln = ln.replace('\n', '')
        for ic, ltr in enumerate(ln):
            themap[lnx, ic] = int(ltr, base=10)
            
    vmap = np.zeros((nrows, ncols), dtype = 'int')
    for ir in range(nrows):
        for ic in range(ncols):
            if ir == 0 or ir == nrows-1 or ic == 0 or ic == ncols-1:
                vmap[ir, ic] = 1
            elif all(themap[ir, ic] > themap[:ir, ic]):
                vmap[ir, ic] = 1
            elif all(themap[ir, ic] > themap[ir, :ic]):
                vmap[ir, ic] = 1
            elif all(themap[ir, ic] > themap[ir+1:, ic]):
                vmap[ir, ic] = 1
            elif all(themap[ir, ic] > themap[ir, ic+1:]):
                vmap[ir, ic] = 1
            print
    print('Total visible {}'.format(np.sum(vmap)))
    vmap2 = np.zeros((nrows, ncols), dtype = 'int')
    for ir in range(nrows):
        for ic in range(ncols):
            visleft = 0
            visright = 0
            visup = 0
            visdown = 0
            for ii in range(ic-1, -1, -1):
                visleft += 1
                if themap[ir, ii] >= themap[ir,ic]:
                    break
            for ii in range(ic+1, ncols):
                visright += 1
                if themap[ir, ii] >= themap[ir,ic]:
                    break
            for ii in range(ir-1, -1, -1):
                visup += 1
                if themap[ii, ic] >= themap[ir,ic]:
                    break
            for ii in range(ir+1, nrows):
                visdown += 1
                if themap[ii, ic] >= themap[ir,ic]:
                    break
            
            #print('{},{}, left {}, right {}, up {}, down {}, prod={}'.format(ir, ic, visleft, visright, visup, visdown,
            #                                                       visleft*visright*visup*visdown))
            vmap2[ir,ic] = visleft * visright * visup * visdown
    print('{}'.format(vmap2.max()))
    
                
    
            
    #_=plt.figure()
    #_=plt.imshow(vmap)

vmap(txtin)

vmap(ptvals)


Total visible 21
8
Total visible 1538
496125


In [158]:
txtin = ['R 4',
        'U 4',
        'L 3',
        'D 1',
        'R 4',
        'D 1',
        'L 5',
        'R 2']

with open('p09_input.txt', 'r') as fid:
    ptvals = fid.readlines()

    
def ropesim(textin, numknots=2):
    headgrid = np.zeros((2000, 2000), dtype = 'int')
    tailgrid = np.zeros((2000, 2000), dtype = 'int')
    hpos = np.zeros((numknots, 2), dtype='int')
    headgrid[hpos[0,0], hpos[0,1]] += 1
    tailgrid[hpos[-1, 0], hpos[-1, 1]] += 1
    hsteps = {'U':np.array([1,0]),
              'D':np.array([-1,0]),
              'R':np.array([0,1]),
              'L':np.array([0,-1])}
    
    for ln in textin:
        ln = ln.replace('\n', '')
        dvs = ln.split(' ')
        for ii in range(int(dvs[1])):
            hpos[0,:] += hsteps[dvs[0]]
            for ik in range(1, numknots):
                htdiff = hpos[ik-1,:] - hpos[ik,:]
                hta = np.abs(htdiff)
                hta[hta==0]=1
                if any(hta>1):
                    hpos[ik,:] += htdiff//hta
                
            
            headgrid[hpos[0,0], hpos[0,1]] += 1
            tailgrid[hpos[-1, 0], hpos[-1, 1]] += 1
            #headgrid[hpos[0], hpos[1]] += 1
            #tailgrid[tpos[0], tpos[1]] += 1
    #_=plt.figure()
    #_=plt.imshow(headgrid)
    #_=plt.figure()
    #_=plt.imshow(tailgrid)
    
    print(np.sum(tailgrid>0))
ropesim(txtin)

ropesim(ptvals)

txty2 = ['R 5', 'U 8', 'L 8', 'D 3', 'R 17', 'D 10', 'L 25', 'U 20']

ropesim(txty2, numknots=10)
ropesim(ptvals, numknots=10)


13
6212
36
2522
